In [3]:
import pandas as pd


In [14]:
import numpy as np
from collections import defaultdict
cluster = pd.read_csv('data/misconception_cluster.csv',header=0)
cluster = cluster['Cluster']
submission = pd.read_csv('data/submission.csv',header=0)
ids = submission['MisconceptionId']

ids = list(map(int, ids[0].split()))
print(ids)

cluster_ids = [cluster[id] for id in ids]
counts = defaultdict(int)
for cluster_id in cluster_ids:
    counts[cluster_id] += 1

counts_ls = [counts[cluster_id] for cluster_id in cluster_ids]

print(cluster_ids)
print(counts_ls)

[1054, 315, 1345, 2488, 1392, 2586, 2306, 1005, 2532, 1700, 77, 1507, 1516, 1084, 706, 1316, 871, 1338, 74, 256, 1941, 483, 2518, 328, 1672, 1963, 2087, 366, 1119, 2181, 2135, 1697, 452, 987, 1336, 528, 1999, 137, 224, 102, 373, 158, 1680, 638, 511, 536, 583, 617, 186, 15, 557, 1722, 1708, 190, 1387, 1444, 2288, 2140, 1496, 969, 2051, 1690, 2022, 1227, 655, 1247, 1936, 981, 1191, 1929, 1563, 1172, 1828, 462, 222, 2131, 602, 1510, 1911, 2383, 1354, 533, 2556, 220, 1534, 487, 1866, 38, 25, 1170, 2237, 2517, 661, 2449, 1765, 1908, 911, 267, 621, 906]
[109, 64, 109, 64, 64, 109, 109, 109, 109, 52, 109, 109, 109, 64, 109, 91, 64, 109, 64, 109, 109, 64, 109, 109, 109, 109, 18, 52, 109, 109, 99, 22, 18, 97, 24, 52, 97, 42, 22, 64, 64, 109, 24, 24, 99, 42, 99, 100, 24, 109, 5, 35, 85, 52, 35, 99, 106, 109, 64, 64, 24, 2, 42, 18, 57, 24, 50, 12, 87, 52, 84, 113, 12, 109, 99, 79, 99, 35, 64, 6, 103, 42, 100, 64, 113, 64, 2, 12, 6, 85, 57, 4, 42, 52, 14, 52, 64, 79, 99, 14]
[23, 15, 23, 15, 15, 2

In [ ]:
import torch

def map_tensor(tensor, mapping_dict):

    # Step 1: Create a mapping tensor
    # Find the maximum key in the dictionary to ensure proper mapping range
    max_key = max(mapping_dict.keys())
    mapping_tensor = torch.zeros(max_key + 1, dtype=torch.int64)
    for key, value in mapping_dict.items():
        mapping_tensor[key] = value

    # Step 2: Apply the mapping using the 2D tensor as an index
    mapped_tensor = mapping_tensor[tensor]
    return mapped_tensor

print("Original Tensor:")
print(tensor)
print("\nMapped Tensor:")
print(mapped_tensor)

Original Tensor:
tensor([[0, 1, 2],
        [3, 4, 5]])

Mapped Tensor:
tensor([[10, 20, 30],
        [40, 50, 60]])


In [20]:
import torch

# Example tensors
candidates = torch.tensor([[0, 1, 2, 3]])  # Shape: (B, 100)
clusters = torch.tensor([[0, 1, 1, 0]])    # Shape: (B, 100)
reranked = torch.tensor([[2, 3, 1, 0]])    # Shape: (B, 100)

B, N = candidates.shape  # B: Batch size, N: Number of elements per row

# Step 1: Create a position mapping for `reranked`
# For each element in `candidates`, find its position in `reranked`
reranked_positions = torch.argsort(reranked, dim=1)

# Step 2: Create a tensor to store final reordered candidates
final_candidates = candidates.clone()

# Step 3: Process each cluster in parallel
for b in range(B):  # Batch-wise processing
    unique_clusters = torch.unique(clusters[b])  # Unique cluster IDs in the row
    for cluster_id in unique_clusters:
        # Mask for the current cluster
        cluster_mask = (clusters[b] == cluster_id)
        cluster_indices = cluster_mask.nonzero(as_tuple=True)[0]

        # Get the `candidates` corresponding to the current cluster
        cluster_candidates = candidates[b, cluster_indices]

        # Get the order of these candidates from `reranked_positions`
        cluster_rerank_indices = torch.argsort(reranked_positions[b, cluster_indices])

        # Reorder the `candidates` within the cluster
        final_candidates[b, cluster_indices] = cluster_candidates[cluster_rerank_indices]

print("Final Reordered Candidates:")
print(final_candidates)


Final Reordered Candidates:
tensor([[3, 2, 1, 0]])


In [3]:
import pandas as pd
df = pd.read_csv('data/all_train.csv', header=0)
df.iloc[2]

QuestionId                                                            2
ConstructId                                                      2774.0
ConstructName                   Calculate the range from a list of data
SubjectId                                                         339.0
SubjectName           Range and Interquartile Range from a List of Data
CorrectAnswer                                                         B
QuestionText          Tom and Katie are discussing the \( 5 \) plant...
AnswerAText                                                   Only\nTom
AnswerBText                                                 Only\nKatie
AnswerCText                                          Both Tom and Katie
AnswerDText                                          Neither is correct
MisconceptionAId                                                 1287.0
MisconceptionBId                                                    NaN
MisconceptionCId                                                